In [ ]:
# 기사 목록
# 상단 불필요한 기사 삭제 필요

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.parse

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화


# Keywords to search
keywords = [
    "검색어",
]
start_date = "2023-08-01"
end_date = "2024-08-31"
collected_urls = set()  # 중복 URL을 저장하기 위한 집합

def fetch_search_results(keyword, start_date, end_date):
    # 각 키워드 검색마다 드라이버 초기화
    driver = webdriver.Chrome(options=chrome_options)
    base_url = "https://www.munhwa.com/search/news"
    search_url = (
        f"{base_url}?word={urllib.parse.quote(keyword, encoding='utf-8')}"
        f"&searchType=NEWS&searchField=&sort=ASC"
        f"&startDate={start_date}&endDate={end_date}"
        f"&codeId=&addWord=&excludeWord="
    )
    driver.get(search_url)
    page = 1  # 페이지 번호 시작
    results = []

    while True:
        paged_url = f"{search_url}&page={page}"
        driver.get(paged_url)
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        articles = soup.select('div.card-list.article-active ul > li')

        if not articles:
            print(f"{keyword} - 더 이상 기사가 없습니다. 수집 종료.")
            break

        print(f"{keyword} - {page}페이지 수집 중...")

        for item in articles:
            try:
                title_tag = item.find("h4", class_="title").find("a")
                title = title_tag.get_text(strip=True) if title_tag else ""
                url = "https:" + title_tag["href"] if title_tag and "href" in title_tag.attrs else ""

                if url in collected_urls:
                    continue
                collected_urls.add(url)

                date_tag = item.find("span", class_="date")
                date = date_tag.get_text(strip=True).split()[0] if date_tag else ""

                writer_tag = item.find("span", class_="writer")
                writer = writer_tag.get_text(strip=True) if writer_tag else ""

                results.append({
                    'media': '문화일보',
                    'title': title,
                    'date': date,
                    'section': '',
                    'article_type': '',
                    'writer': writer,
                    'page': '',
                    'url': url,
                    'content': ''
                })
            except Exception as e:
                print(f"Error processing article: {e}")

        page += 1  # 다음 페이지로 이동


    # 검색이 끝난 후 드라이버 종료
    driver.quit()
    return results

all_results = []
for keyword in keywords:
    # 각 키워드와 기간을 매개변수로 전달
    results = fetch_search_results(keyword, start_date, end_date)
    all_results.extend(results)
    df = pd.DataFrame(all_results)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(f'1문화일보_{start_date}~{end_date}.xlsx', index=False)



In [ ]:
# 본문 추출

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, 'error_log.txt')  # 오류 로그 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문과 서브 타이틀을 스크랩하는 함수 정의
def get_content_and_subtitle_from_url(url):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 서브 타이틀 추출 (b.sub_title)
        subtitle = ""
        article_body = soup.find('div', id='article-body')
        if article_body:
            # 서브타이틀 추출 및 제거
            subtitle_tag = article_body.find('b')
            if subtitle_tag:
                subtitle = subtitle_tag.get_text(separator='\n', strip=True).strip('"')
                subtitle_tag.decompose()

            # 이미지나 불필요한 태그 제거 (선택적으로 유지)
            for caption in article_body.find_all('figure', class_='article-img'):
                caption.decompose()    

        # 본문 추출
            content = article_body.get_text(separator='\n', strip=True).strip('"')
            content = '\n'.join(
    line for line in content.split('\n') if '@munhwa.com' not in line
    )
        else:
            content = ""

        return content, subtitle
    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None, None


# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문과 서브 타이틀 내용을 스크랩하여 content와 sub-title 컬럼에 저장합니다.
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle = get_content_and_subtitle_from_url(row['url'])

    if content is None:
        error_log.append(row['url'])
        continue  # 오류가 발생한 경우 다음 URL로 넘어감

    if content.startswith('='):
        content = "'" + content

    df.at[index, 'content'] = content
    df.at[index, 'sub-title'] = subtitle  # 서브 타이틀 컬럼 추가

    # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 불필요한 항목을 제외한 후 엑셀 파일로 저장
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")
